**Realated Work**
* Palestine Data from API [corona.ps](http://corona.ps/)
* Global Data, Click [here](https://www.kaggle.com/imdevskp/corona-virus-report)
* For Analysis and Prediction on Coronavirus(Italy), Click [here](https://www.kaggle.com/vanshjatana/analysis-and-prediction-on-coronavirus-italy)
*  For Analysis and Prediction on Coronavirus(South-Korea), Click [here](https://www.kaggle.com/vanshjatana/analysis-on-coronavirus)
*  For Machine Learning on Cornovirus, Click [here](https://www.kaggle.com/vanshjatana/machine-learning-on-coronavirus)
*  For report on Coronavirus, Click [here](https://www.researchgate.net/publication/339738108_Analysis_On_Coronavirus)


In [ ]:
import datetime
import numpy as np
import pandas as pd 
import seaborn as sns
import plotly.offline as py
import plotly.express as px
from fbprophet import Prophet
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.cluster import KMeans
from datetime import date, timedelta
from plotly.subplots import make_subplots
from statsmodels.tsa.arima_model import ARIMA
from fbprophet.plot import plot_plotly, add_changepoints_to_plot

In [ ]:
data=pd.read_csv("../input/corona-virus-palestine/corona-virus-palestine.csv")
globalData=pd.read_csv("../input/corona-virus-report/covid_19_clean_complete.csv")

# Cases in Palestine

In [ ]:
import folium
worldmap = folium.Map(location=[32.072814, 34.777636 ], zoom_start=8,tiles='Stamen Toner')
for Lat, Long, state in zip(data['Lat'], data['Long'],data['case_location']): folium.CircleMarker([Lat, Long], radius=5, color='red', popup =('State: ' + str(state) + '<br>'), fill_color='red', fill_opacity=0.7 ).add_to(worldmap)
worldmap

In [ ]:
locations=data.groupby(['case_location','case_condition'])
tmpdata=[]
for key,case in locations:
    v={'Location':key[0],key[1]:case['case_condition'].count()}
    tmpdata.append(v)

locationData=pd.DataFrame(tmpdata)
locationData.columns = ['Location','Recovered', 'Confiremed', 'Death'] 
locationData=locationData.fillna(0)
locationData=locationData.groupby(['Location'])
tmpdata=[]
for key,_data in locationData:
    v={'Location':key,'Recovered':_data['Recovered'].sum(),'Confiremed':_data['Confiremed'].sum(),'Death':_data['Death'].sum()}
    tmpdata.append(v)
locationData=pd.DataFrame(tmpdata)
print(locationData)

# **Data Analysis**

> Gender

In [ ]:
# Data to plot
plotData = data.groupby(["case_gender"])["case_gender"].count()

# Plot
plt.axis('equal');
plt.pie(plotData, labels=plotData.index,autopct='%1.1f%%',colors=['pink','lightskyblue']);
plt.show()

> Age
> 
*Infants (0-2) years | 
Child (2-18) years | 
Adults (18-45) years | 
Seniors >45 years*

In [ ]:
# Data to plot
bins= [0,2,18,45,100]
labels = ['Infant','Child', 'Adult','Senior']
data['AgeGroup'] = pd.cut(data['case_age'], bins=bins, labels=labels, right=True)

plotData = data.groupby(["AgeGroup"])["AgeGroup"].count()
# Plot
plt.axis('equal');
plt.pie(plotData, labels=plotData.index,autopct='%1.1f%%');
plt.show()

> Quarantine

In [ ]:
# Data to plot
plotData = data.groupby(["case_quarantine"])["case_quarantine"].count()

# Plot
plt.axis('equal');
plt.pie(plotData, labels=plotData.index,autopct='%1.1f%%');
plt.show()

> Source of infection

In [ ]:
# Data to plot
plotData = data.groupby(["case_source_of_infection"])["case_source_of_infection"].count()

# Plot
plt.axis('equal');
plt.pie(plotData, labels=plotData.index,autopct='%1.1f%%');
plt.show()

# **Palestine Covid-19 Status**

In [ ]:
palestine_data = globalData[globalData['Country/Region']=='West Bank and Gaza']
palestine_data['Date'] = pd.to_datetime(palestine_data['Date'])
palestine_data = palestine_data.sort_values(by='Date')
f, ax = plt.subplots(figsize=(23,10))
ax=sns.scatterplot(x="Date", y="Deaths", data=palestine_data, color="red",label = "Death")
ax=sns.scatterplot(x="Date", y="Recovered", data=palestine_data, color="green",label = "Recovery")
ax=sns.scatterplot(x="Date", y="Confirmed", data=palestine_data, color="blue",label = "Confirm")
plt.plot(palestine_data.Date,palestine_data.Deaths,zorder=1,color="red")
plt.plot(palestine_data.Date,palestine_data.Recovered,zorder=1,color="green")
plt.plot(palestine_data.Date,palestine_data.Confirmed,zorder=1,color="blue")
ax.set_xlim([datetime.date(2020, 3, 1), palestine_data['Date'].iloc[-1]])
ax.set_ylim([-1, max(palestine_data['Confirmed']) + 20])
f.autofmt_xdate()

# **Covid-19 Growth Rate in Palestine**

In [ ]:
growth_rate = (palestine_data['Confirmed'] / palestine_data['Confirmed'].shift()[-4:]).mean()
print('Growth Rate: ', growth_rate)
print('Yesterday\'s (%s) confirmed: ' % (palestine_data.Date.iloc[-1].strftime('%Y.%m.%d')), palestine_data['Confirmed'].iloc[-1])
print('Expected Tommorw: ', palestine_data['Confirmed'].iloc[-1] * (palestine_data['Confirmed'] / palestine_data['Confirmed'].shift()[-4:]).mean())